<a href="https://colab.research.google.com/github/Senthu08/Resume-Screening-in-Google-colab/blob/main/FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

def uerd_encode(cover_image, secret_data):
    """Encodes secret data into a cover image using the UERD steganography algorithm."""
    secret_data_len = len(secret_data) * 8
    cover_image_width, cover_image_height, _ = cover_image.shape
    stego_image = np.zeros((cover_image_width, cover_image_height, 3), dtype=np.uint8)

    for i in range(secret_data_len):
        pixel_index, bit_position = divmod(i, 3)

        # Convert pixel to float32 before applying DCT
        pixel_float32 = cover_image[pixel_index].astype(np.float32)
        dct_coefficients = cv2.dct(pixel_float32)

        threshold = np.mean(dct_coefficients)
        if secret_data[i // 8] == '1':
            dct_coefficients[0, bit_position] += 1  # Only modify the specified DCT coefficient
        else:
            dct_coefficients[0, bit_position] -= 1  # Only modify the specified DCT coefficient

        # Convert back to float32 after applying IDCT
        stego_image[pixel_index] = cv2.idct(dct_coefficients).astype(np.uint8)

    return stego_image

def uerd_decode(stego_image):
    """Decodes secret data from a stego image using the UERD steganography algorithm."""
    secret_data = ""
    secret_data_len = len(stego_image) * 3 * 8
    num_pixels = len(stego_image)

    for i in range(secret_data_len):
        pixel_index, bit_position = divmod(i, 3)

        # Ensure pixel_index is within the valid range
        if pixel_index >= num_pixels:
            break
        # Convert pixel to float32 before applying DCT
        pixel_float32 = stego_image[pixel_index].astype(np.float32)
        dct_coefficients = cv2.dct(pixel_float32)

        threshold = np.mean(dct_coefficients)
        secret_data += str(int(dct_coefficients[0, bit_position] > threshold))

    return secret_data

def representation_data(data):
    # Check for non-printable characters and their positions
    non_printable_positions = [(i, ord(char)) for i, char in enumerate(data) if not char.isprintable()]

    if non_printable_positions:
        print("Non-printable characters found at positions:")
        for position, char_code in non_printable_positions:
            print(f"Position: {position}, Character Code: {char_code}")

    # Check the length of the data
    print(f"Data Length: {len(data)}")

    # Print the data as both ASCII and binary
    print("ASCII Representation:")
    for char in data:
        print(f"{char} ({ord(char)})")

    print("Binary Representation:")
    binary_data = ' '.join(format(ord(char), '08b') for char in data)
    print(binary_data)

# Example data (replace this with your actual data)
data = "LOGANATHAVISHNUBALAJI"

# Encode the data
cover_image = cv2.imread("/content/download (1).jpg")  # Replace with the path to your cover image
stego_image = uerd_encode(cover_image, data)

# Save the stego image
cv2.imwrite("stego_image.png", stego_image)

# Decode the data from the stego image
decoded_data = uerd_decode(stego_image)

# Debug the decoded data
representation_data(decoded_data)


Data Length: 21
ASCII Representation:
L (76)
O (79)
G (71)
A (65)
N (78)
A (65)
T (84)
H (72)
A (65)
V (86)
I (73)
S (83)
H (72)
N (78)
U (85)
B (66)
A (65)
L (76)
A (65)
J (74)
I (73)
Binary Representation:
01001100 01001111 01000111 01000001 01001110 01000001 01010100 01001000 01000001 01010110 01001001 01010011 01001000 01001110 01010101 01000010 01000001 01001100 01000001 01001010 01001001


NSF5

In [ ]:
import numpy as np
from scipy.fftpack import dct

# Install OpenCV.
!pip install opencv-python

# Import OpenCV.
import cv2

def nsf5_steganalysis(image):
  """
  Performs nsF5 steganalysis on the given image.

  Args:
    image: A numpy array representing the image to be analyzed.

  Returns:
    A float value representing the steganalysis score, where higher scores indicate a higher likelihood of the image being steganographic.
  """

  # Compute the DCT coefficients of the image.
  dct_coeffs = dct(image)

  # Compute the mean and standard deviation of the DCT coefficients.
  mean = np.mean(dct_coeffs)
  std = np.std(dct_coeffs)

  # Compute the nsF5 steganalysis score.
  score = (mean - 128) / std

  return score


images = []
for i in range(5):
  image = cv2.imread('/content/stego_image.png')
  images.append(image)

# Perform nsF5 steganalysis on each image.
steganalysis_scores = []
for image in images:
  score = nsf5_steganalysis(image)
  steganalysis_scores.append(score)

# Print the steganalysis scores for all 5 images.
print("Steganalysis scores:")
for i in range(5):
  print(f"Image {i}: {steganalysis_scores[i]}")

Steganalysis scores:
Image 0: -0.18328842977857157
Image 1: -0.18328842977857157
Image 2: -0.18328842977857157
Image 3: -0.18328842977857157
Image 4: -0.18328842977857157


SRM


In [ ]:
def srm_steganalysis(image):
    """
    Performs SRM steganalysis on the given image.

    Args:
        image: A numpy array representing the image to be analyzed.

    Returns:
        A float value representing the steganalysis score, where higher scores indicate a higher likelihood of the image being steganographic.
    """
    # Convert the image to grayscale if it's not already.
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute the spatial statistics using the SRM algorithm.
    srm_result = srm_features(image)

    # Calculate a steganalysis score based on the SRM features.
    score = calculate_srm_score(srm_result)

    return score


In [ ]:
import numpy as np

def srm_features(image):
    # This is a simplified example. You may want to implement a more sophisticated SRM feature extraction.
    height, width = image.shape
    features = np.zeros((height, width, 5))  # You can adjust the number of features as needed.

    for i in range(height):
        for j in range(width):
            # Compute some simple spatial features (e.g., local mean, variance, etc.).
            local_mean = np.mean(image[i:i+3, j:j+3])
            local_variance = np.var(image[i:i+3, j:j+3])
            # Store these features in the result.
            features[i, j, 0] = local_mean
            features[i, j, 1] = local_variance

    return features

def calculate_srm_score(srm_result):
    # Calculate the steganalysis score based on the extracted SRM features.
    # This is a simplified example and can be improved for better results.
    return np.mean(srm_result)

# Load the image from Google Drive.
image = cv2.imread('/content/stego_image.png')

# Perform SRM steganalysis on the image.
srm_result = srm_steganalysis(image)

# Calculate the steganalysis score based on SRM features.
score = calculate_srm_score(srm_result)

# Print the steganalysis score.
print("Steganalysis score:",score)

Steganalysis score: 87.12664472967971


HDPP

In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def hppd_steganalysis(image):
    # Implement the HPPD steganalysis algorithm.
    # This should include feature extraction and scoring.
    # Replace this with your actual implementation.

    # For illustration, let's assume a random score between 0 and 1.
    return np.random.random()

def load_labeled_dataset():
    # Load your labeled dataset, including image paths and corresponding labels (1 for steganographic, 0 for non-steganographic).
    # You should replace this with your own dataset loading code.

    # Example dataset:
    image_paths = ["steg_image1.png", "steg_image2.png", "non_steg_image1.png", "non_steg_image2.png"]
    labels = [1, 1, 0, 0]

    return image_paths, labels

def calculate_accuracy(predictions, labels):
    # Calculate accuracy based on predictions and ground truth labels.
    return accuracy_score(labels, [1 if p > 0.5 else 0 for p in predictions])

# Load the labeled dataset.
image_paths, labels = load_labeled_dataset()

# Initialize lists to store steganalysis scores and ground truth labels.
scores = []
ground_truth = []

# Perform HPPD steganalysis on each image and collect the scores and labels.
for image_path, label in zip(image_paths, labels):
    image = cv2.imread(image_path)
    score = hppd_steganalysis(image)
    scores.append(score)
    ground_truth.append(label)

# Calculate accuracy.
accuracy = calculate_accuracy(scores, ground_truth)

# Print the steganalysis accuracy.
print("Steganalysis Accuracy:", accuracy)


Steganalysis Accuracy: 0.5
